In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from features.samplers import PairSampler, TripletSampler
from features.settings import Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from keras.datasets import mnist, fashion_mnist, cifar10, cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_other = input_data.read_data_sets('MNIST_data', one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
def test_model(model, settings, num_features, num_steps, lr, keep_prob, reg):

    x = tf.placeholder(tf.float32, shape=[None, 784])
    y = tf.placeholder(tf.int64, shape=[None])
    dropout = tf.placeholder(tf.float32)
    m = model(x, y, num_features, settings, lr, reg, dropout)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training // num_features: %g, lr: %g, reg: %g"%(num_features, lr, reg))
        train_time = time.time()
        for step in range(num_steps):
            x_, y_ = train.sample(100)
            sess.run(m.optimize, feed_dict={x:x_, y:y_, dropout:keep_prob})     
            if step % 1000 == 0:
                train_loss = sess.run(m.loss, feed_dict={x:x_, y:y_, dropout: 1.0})                
                train_error = 1 - sess.run(m.acc, feed_dict={x:x_, y:y_, dropout: 1.0})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, train_error))  
        train_time = time.time() - train_time
        print("end training // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = test.sample(500)
        test_error = 1 - sess.run(m.acc, feed_dict={x:x_, y:y_, dropout:1.0})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        svc = LinearSVC(random_state=0)
        features_train = sess.run(m.features, feed_dict={x:x_train, y:y_train, dropout: 1.0})
        svc.fit(features_train, y_train)
        features_test = sess.run(m.features, feed_dict={x:x_test, y:y_test, dropout: 1.0})
        print("classification accuracy: {:.4f}".format(svc.score(features_test, y_test)))

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape((60000, -1))/255.0, x_test.reshape((10000, -1))/255.0
train, test = TripletSampler((x_train, y_train)), TripletSampler((x_test, y_test))

test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
           num_features=10, num_steps=20000, lr=0.0005, keep_prob=0.8, reg=0)

begin training // num_features: 10, lr: 0.0005, reg: 0
	step 0: train loss 1.0027, train error 0.52
	step 1000: train loss 0.0299508, train error 0.02
	step 2000: train loss 0.018441, train error 0
	step 3000: train loss 0.0396671, train error 0.02
	step 4000: train loss 0.018127, train error 0.00999999
	step 5000: train loss 0, train error 0
	step 6000: train loss 0.0105485, train error 0
	step 7000: train loss 0, train error 0
	step 8000: train loss 0.0235283, train error 0.00999999
	step 9000: train loss 0.00892892, train error 0
	step 10000: train loss 0, train error 0
	step 11000: train loss 0.00196601, train error 0
	step 12000: train loss 0, train error 0
	step 13000: train loss 0.0217008, train error 0.00999999
	step 14000: train loss 0, train error 0
	step 15000: train loss 0, train error 0
	step 16000: train loss 0.000532984, train error 0
	step 17000: train loss 0, train error 0
	step 18000: train loss 0, train error 0
	step 19000: train loss 0.003248, train error 0
end trai

In [10]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape((60000, -1))/255.0, x_test.reshape((10000, -1))/255.0
train, test = PairSampler((x_train, y_train)), PairSampler((x_test, y_test))

test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
           num_features=10, num_steps=30000, lr=0.0001, keep_prob=0.8, reg=0)

begin training // num_features: 10, lr: 0.0001, reg: 0
	step 0: train loss 2.18447, train error 0.89
	step 1000: train loss 1.00494, train error 0.86
	step 2000: train loss 0.90664, train error 0.5
	step 3000: train loss 0.796032, train error 0.39
	step 4000: train loss 0.779033, train error 0.3
	step 5000: train loss 0.580968, train error 0.23
	step 6000: train loss 0.508518, train error 0.18
	step 7000: train loss 0.563187, train error 0.22
	step 8000: train loss 0.765119, train error 0.26
	step 9000: train loss 0.556815, train error 0.16
	step 10000: train loss 0.316389, train error 0.1
	step 11000: train loss 0.506035, train error 0.15
	step 12000: train loss 0.406031, train error 0.12
	step 13000: train loss 0.246979, train error 0.09
	step 14000: train loss 0.318502, train error 0.11
	step 15000: train loss 0.39917, train error 0.12
	step 16000: train loss 0.271222, train error 0.06
	step 17000: train loss 0.326984, train error 0.1
	step 18000: train loss 0.188281, train error 0.